# Polars Library Python for data analysis


### Reading Excel file


In [2]:
import polars as pl

In [3]:
df = pl.read_excel("Sample - Superstore.xls", sheet_name="Orders")
df.head()

Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
i64,str,date,date,str,str,str,str,str,str,str,i64,str,str,str,str,str,f64,i64,f64,f64
1,"""CA-2016-152156""",2016-11-08,2016-11-11,"""Second Class""","""CG-12520""","""Claire Gute""","""Consumer""","""United States""","""Henderson""","""Kentucky""",42420,"""South""","""FUR-BO-10001798""","""Furniture""","""Bookcases""","""Bush Somerset Collection Bookc…",261.96,2,0.0,41.9136
2,"""CA-2016-152156""",2016-11-08,2016-11-11,"""Second Class""","""CG-12520""","""Claire Gute""","""Consumer""","""United States""","""Henderson""","""Kentucky""",42420,"""South""","""FUR-CH-10000454""","""Furniture""","""Chairs""","""Hon Deluxe Fabric Upholstered …",731.94,3,0.0,219.582
3,"""CA-2016-138688""",2016-06-12,2016-06-16,"""Second Class""","""DV-13045""","""Darrin Van Huff""","""Corporate""","""United States""","""Los Angeles""","""California""",90036,"""West""","""OFF-LA-10000240""","""Office Supplies""","""Labels""","""Self-Adhesive Address Labels f…",14.62,2,0.0,6.8714
4,"""US-2015-108966""",2015-10-11,2015-10-18,"""Standard Class""","""SO-20335""","""Sean O'Donnell""","""Consumer""","""United States""","""Fort Lauderdale""","""Florida""",33311,"""South""","""FUR-TA-10000577""","""Furniture""","""Tables""","""Bretford CR4500 Series Slim Re…",957.5775,5,0.45,-383.031
5,"""US-2015-108966""",2015-10-11,2015-10-18,"""Standard Class""","""SO-20335""","""Sean O'Donnell""","""Consumer""","""United States""","""Fort Lauderdale""","""Florida""",33311,"""South""","""OFF-ST-10000760""","""Office Supplies""","""Storage""","""Eldon Fold 'N Roll Cart System""",22.368,2,0.2,2.5164


In [4]:
df.schema

Schema([('Row ID', Int64),
        ('Order ID', String),
        ('Order Date', Date),
        ('Ship Date', Date),
        ('Ship Mode', String),
        ('Customer ID', String),
        ('Customer Name', String),
        ('Segment', String),
        ('Country', String),
        ('City', String),
        ('State', String),
        ('Postal Code', Int64),
        ('Region', String),
        ('Product ID', String),
        ('Category', String),
        ('Sub-Category', String),
        ('Product Name', String),
        ('Sales', Float64),
        ('Quantity', Int64),
        ('Discount', Float64),
        ('Profit', Float64)])

### Monthly profit


In [5]:
a = (
    df.group_by(
        pl.col("Order Date").dt.year().alias("order_year"),
        pl.col("Order Date").dt.month().alias("order_month"),
    )
    .agg(pl.col("Profit").sum().round(2).alias("total_profit"))
    .sort([pl.col("order_year"), pl.col("order_month")])
)

In [6]:
a

order_year,order_month,total_profit
i32,i8,f64
2014,1,2450.19
2014,2,862.31
2014,3,498.73
2014,4,3488.84
2014,5,2738.71
…,…,…
2017,8,9040.96
2017,9,10991.56
2017,10,9275.28


In [7]:
a.write_csv("monthly_profit.csv")

In [8]:
import altair as alt

In [9]:
a.plot.line(
    x="order_month",
    y="total_profit",
    color=alt.Color(
        "order_year:N",
        scale=alt.Scale(scheme="category10"),
        legend=alt.Legend(title="Year"),
    ),
).properties(width=1000, height=600)

alt.Chart(...)

# Top 5 states with most profit


In [10]:
b = (
    df.group_by(pl.col("State"))
    .agg(pl.col("Profit").sum().round(2).alias("total_profit"))
    .sort(pl.col("total_profit"), descending=True)
    .head()
)

In [11]:
b

State,total_profit
str,f64
"""California""",76381.39
"""New York""",74038.55
"""Washington""",33402.65
"""Michigan""",24463.19
"""Virginia""",18597.95


In [12]:
# Bar chart
bar_chart = b.plot.bar(x=alt.X("State", sort="-y"), y="total_profit").properties(
    width=800, height=400, title="Top 5 states by Profit"
)

# Text labels on top of bars
text = bar_chart.mark_text(
    align="center", baseline="bottom", dy=-5  # Adjust position above bars
).encode(text="total_profit")

bar_chart + text

alt.LayerChart(...)

# Region and Segment wise sales


In [13]:
c = df.group_by(pl.col("Region"), pl.col("Segment")).agg(
    pl.col("Sales").sum().round(2).alias("total_sales")
)

In [14]:
c

Region,Segment,total_sales
str,str,f64
"""West""","""Consumer""",362880.77
"""West""","""Home Office""",136721.78
"""Central""","""Corporate""",157995.81
"""West""","""Corporate""",225855.27
"""Central""","""Home Office""",91212.64
…,…,…
"""South""","""Consumer""",195580.97
"""East""","""Home Office""",127463.73
"""East""","""Corporate""",200409.35


In [15]:
region_order = (
    df.group_by(pl.col("Region"))
    .agg(pl.col("Sales").sum().round(2).alias("total_sales"))
    .sort(pl.col("total_sales"), descending=True)
)
region_order

Region,total_sales
str,f64
"""West""",725457.82
"""East""",678781.24
"""Central""",501239.89
"""South""",391721.91


In [16]:
region_order = region_order["Region"].to_list()
region_order

['West', 'East', 'Central', 'South']

In [17]:
chart = c.plot.bar(
    x=alt.X("Region:N", sort=region_order), y="total_sales:Q", color="Segment:N"
).properties(width=800, height=400, title="Region and Segment wise Total Sales")

In [18]:
chart

alt.Chart(...)

In [19]:
type(chart)

altair.vegalite.v5.api.Chart

# Stacked bar chart custom defifnition


In [20]:
def stacked_bar_chart(
    df: pl.DataFrame, x: str, y: str, color: str
) -> tuple[pl.DataFrame, alt.Chart]:
    a = df.group_by(pl.col(x), pl.col(color)).agg(
        pl.col(y).sum().round(2).alias(f"total_{y}")
    )
    order = (
        df.group_by(pl.col(x))
        .agg(pl.col(y).sum().round(2).alias(f"total_{y}"))
        .sort(pl.col(f"total_{y}"), descending=True)
    )
    order = order[x].to_list()
    chart = a.plot.bar(
        x=alt.X(f"{x}:N", sort=order), y=f"total_{y}:Q", color=f"{color}:N"
    ).properties(width=800, height=400, title=f"{x} and {color} wise {y}")
    return a, chart

In [21]:
df.columns

['Row ID',
 'Order ID',
 'Order Date',
 'Ship Date',
 'Ship Mode',
 'Customer ID',
 'Customer Name',
 'Segment',
 'Country',
 'City',
 'State',
 'Postal Code',
 'Region',
 'Product ID',
 'Category',
 'Sub-Category',
 'Product Name',
 'Sales',
 'Quantity',
 'Discount',
 'Profit']

In [22]:
a, chart = stacked_bar_chart(df, x="Category", y="Quantity", color="Segment")

In [23]:
a

Category,Segment,total_Quantity
str,str,i64
"""Office Supplies""","""Corporate""",7018
"""Technology""","""Corporate""",2095
"""Technology""","""Home Office""",1247
"""Furniture""","""Home Office""",1367
"""Furniture""","""Consumer""",4166
"""Technology""","""Consumer""",3597
"""Furniture""","""Corporate""",2495
"""Office Supplies""","""Consumer""",11758
"""Office Supplies""","""Home Office""",4130


In [24]:
chart

alt.Chart(...)

In [25]:
b, chart = stacked_bar_chart(df, x="Sub-Category", y="Profit", color="Region")

In [26]:
b

Sub-Category,Region,total_Profit
str,str,f64
"""Machines""","""Central""",-1486.07
"""Envelopes""","""Central""",1777.53
"""Appliances""","""East""",8391.41
"""Art""","""Central""",1195.16
"""Storage""","""Central""",1969.84
…,…,…
"""Copiers""","""East""",17022.84
"""Phones""","""South""",10767.28
"""Paper""","""South""",5947.06


In [27]:
chart

alt.Chart(...)

# Monthly order profit


In [30]:
c = (
    df.group_by(
        pl.date(
            pl.col("Order Date").dt.year(), pl.col("Order Date").dt.month(), 1
        ).alias("order_month")
    )
    .agg(pl.col("Profit").sum().round(2).alias("total_profit"))
    .sort(pl.col("order_month"))
)

In [31]:
c

order_month,total_profit
date,f64
2014-01-01,2450.19
2014-02-01,862.31
2014-03-01,498.73
2014-04-01,3488.84
2014-05-01,2738.71
…,…
2017-08-01,9040.96
2017-09-01,10991.56
2017-10-01,9275.28


In [33]:
c.plot.line(x="order_month", y="total_profit").properties(
    width=800, height=400, title="Monthly Profit"
)

alt.Chart(...)